In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import os
import warnings
import functools
import seaborn as sns
import scipy.stats
import anndata
import dandelion as ddl
sc.logging.print_header()

warnings.filterwarnings('ignore')
os.chdir('/lustre/scratch117/cellgen/team297/kt16/Ziad/scanpy')

scanpy==1.7.1 anndata==0.7.5 umap==0.5.1 numpy==1.19.4 scipy==1.6.0 pandas==1.2.3 scikit-learn==0.23.2 statsmodels==0.12.1 python-igraph==0.8.3 leidenalg==0.8.3


In [2]:
rna = sc.read_h5ad('h5ad/adata_soupx_trans_cite_rna.h5ad')

In [3]:
from tools import returnDEres

In [4]:
from collections import defaultdict
adata = defaultdict(dict)

In [5]:
import numpy as np
import math
sign = lambda x: math.copysign(1, x)

In [6]:
sorted(list(set(rna.obs['fine_clustering'])))

['B_naive',
 'B_non-switched_memory',
 'B_switched_memory',
 'CD16neg_NK',
 'CD16pos_NK',
 'CD16pos_SIGLEC7_NK',
 'CD4_Tcm',
 'CD4_Tem',
 'CD4_Th2',
 'CD4_Tnaive',
 'CD4_Treg',
 'CD8_Tem',
 'CD8_Temra',
 'CD8_Tnaive',
 'Classical_mono',
 'Erythrocyte',
 'HSC',
 'Intermediate_mono',
 'MAIT',
 'Megakaryocyte',
 'NKT',
 'Non-classical_mono',
 'Non-classical_mono_C1Q+',
 'Plasmablast',
 'Proliferating_lymphocyte',
 'Vd1_gdT',
 'Vd2_gdT',
 'cDC1',
 'cDC2',
 'doublets',
 'pDC']

In [7]:
rna = rna[~(rna.obs['fine_clustering'].isin(['doublets', 'Erythrocyte', 'Megakaryocyte', 'Proliferating_lymphocyte']))].copy()

In [8]:
rna.var

,gene_ids,feature_types,mt,n_cells_by_counts,mean_counts,pct_dropout_by_counts,total_counts,n_cells,highly_variable,means,dispersions,dispersions_norm,mean,std
DVL1,ENSG00000107404,Gene Expression,False,1142,0.004824,99.515997,1138.158813,963,True,0.027555,2.371267,0.552155,-1.213062e-11,0.126689
PGD,ENSG00000142657,Gene Expression,False,42932,0.244466,81.804543,57681.531250,38529,True,0.907115,2.669630,1.252420,4.603412e-11,0.802019
PLOD1,ENSG00000083444,Gene Expression,False,9541,0.042212,95.956330,9959.924805,8519,True,0.236344,2.423540,0.678049,1.882275e-11,0.382022
TNFRSF1B,ENSG00000028137,Gene Expression,False,62818,0.417973,73.376450,98620.328125,55728,True,1.203213,2.574883,0.585680,-3.688341e-11,0.960224
PADI2,ENSG00000117115,Gene Expression,False,1819,0.007941,99.229071,1873.632812,1614,True,0.047854,2.367778,0.543752,1.216257e-12,0.167295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
COL6A2,ENSG00000142173,Gene Expression,False,4713,0.022024,98.002534,5196.504395,4169,True,0.142589,2.353597,0.509599,-1.519736e-11,0.294109
S100B,ENSG00000160307,Gene Expression,False,3008,0.027139,98.725148,6403.381348,2692,True,0.153530,3.164904,2.463557,2.913311e-11,0.277133
MT-ND5,ENSG00000198786,Gene Expression,True,201025,3.359398,14.801504,792646.500000,175509,True,2.938381,2.814749,0.910557,-4.620387e-11,1.096896
MT-ND6,ENSG00000198695,Gene Expression,True,97730,0.851830,58.580032,200988.421875,81893,True,1.706805,3.066679,0.815468,-3.375136e-11,1.118917


In [9]:
for x in list(set(rna.obs['fine_clustering'])):
    adata[x] = rna[rna.obs['fine_clustering'] == x].raw.to_adata()
    sc.pp.filter_genes(adata[x], min_cells = 0)

In [10]:
for x in adata:
    try:
        sc.tl.rank_genes_groups(adata[x], groupby = 'treatment_group_1', reference = 'untreated', method = 'wilcoxon', n_genes = 30000, pts=True)
    except:
        pass

In [11]:
res1 = defaultdict(dict)
res2 = defaultdict(dict)

for x in adata:
    try:
        res1[x] = returnDEres(adata[x], column = '1.5MIU', remove_mito_ribo = False)
        res2[x] = returnDEres(adata[x], column = '2.5MIU', remove_mito_ribo = False)
        
        res1[x] = res1[x][(res1[x]['pts']>=.1)]
        res1[x]['sign'] = [sign(y) for y in res1[x]['logfoldchanges']]
        res1[x]['rank'] = -np.log10(res1[x]['pvals'])*res1[x]['sign']
        
        res2[x] = res2[x][(res2[x]['pts']>=.1)]
        res2[x]['sign'] = [sign(y) for y in res2[x]['logfoldchanges']]
        res2[x]['rank'] = -np.log10(res2[x]['pvals'])*res2[x]['sign']
        
        res1[x] = res1[x].reset_index()
        res1[x] = res1[x][['index', 'rank']]
        
        res2[x] = res2[x].reset_index()
        res2[x] = res2[x][['index', 'rank']]
    except:
        pass

In [12]:
if not os.path.exists('DEG/1.5MIU_vs_untreated'):
    os.makedirs('DEG/1.5MIU_vs_untreated')
if not os.path.exists('DEG/2.5MIU_vs_untreated'):
    os.makedirs('DEG/2.5MIU_vs_untreated')

In [13]:
for x in res1:
    res1[x].to_csv('DEG/1.5MIU_vs_untreated/'+x+'.csv')
for x in res2:
    res2[x].to_csv('DEG/2.5MIU_vs_untreated/'+x+'.csv')